In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import pickle

In [2]:
df = pd.read_csv('../../data/raw/zillow_data.csv')

In [27]:
def one_zipcode(df, index):
    """This function pulls the data for one zipcode at a time and retuns a DataFrame for using in Prophet."""
    series = df.iloc[index]
    series_data = series.iloc[7:]
    df_series = pd.DataFrame(series_data.values, index = series_data.index, columns = ['y'])
    df_series.index = pd.to_datetime(df_series.index, yearfirst = True, format = '%Y-%m')
    df_series['ds'] = df_series.index
    df_series.reset_index(drop = True, inplace = True)
    df_series['y'] = df_series['y'].astype('int64', errors='ignore')
    return df_series

In [4]:
def get_zipcode(df, index):
    """This function extracts the zipcode from the dataframe at the index. Useful in constructing data dictionanry
    later."""
    zipcode = df['RegionName'].iloc[index]
    return zipcode

In [5]:
def Prophet_analysis(df):
    """This function instantiates a Prophet model, fits it to the DataFrame, and predicts values which are returned in 
    a forecast Dataframe."""
    m = Prophet(seasonality_mode='multiplicative', interval_width=0.95)
    m.fit(df)
    future = m.make_future_dataframe(60, freq = 'M')
    forecast = m.predict(future)
    return forecast

In [6]:
def most_recent_value(df):
    '''
    Finds the most recent median home value for use in future percent calculation.
    '''
    i = -1
    while np.isnan(df.iloc[i]['y']):
        i -= 1

    return df.iloc[i]['y']

In [7]:
def combine_dataframes(df, forecast):
    """This function combines the original dataframe with data from one zipcode and the forecasted values from the
    Prophet analysis into a single DataFrame. It also adds new columns for percent change from the last value in
    the original dataframe and the estimated gross profit."""
    forecast.rename(columns={'yhat': 'y'}, inplace = True)
    forecast_subset = forecast[forecast['ds']>'2018-04-01'][['ds','y', 'yhat_lower', 'yhat_upper']]
    df_appended = df.append(forecast_subset, sort = True)
    df_appended['pct_change'] = ((df_appended['y']-most_recent_value(df))/most_recent_value(df))*100
    df_appended['gross_profit'] = (df_appended['y']-most_recent_value(df))
    return df_appended

In [8]:
def make_dict(df, zipcode):
    """This function returns a summary dictionary from the combined dataframe for a given zipcode. This dictionary
    can be appended to a list that will be used to make a final dataframe."""
    pct_change_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['pct_change'].values[0]
    gross_profit_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['gross_profit'].values[0]
    pct_change_5year = df.loc[(df['ds']>='2023-03-31')]['pct_change'].values[0]
    gross_profit_5year = df.loc[(df['ds']>='2023-03-31')]['gross_profit'].values[0]
    estimate_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['y'].values[0]
    upper_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_upper'].values[0]
    lower_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_lower'].values[0]
    estimate_5year = df.loc[(df['ds']>='2023-03-31')]['y'].values[0]
    upper_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_upper'].values[0]
    lower_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_lower'].values[0]
    zipcode_dict = {'Zipcode': zipcode, 
                    'estimate_1year': estimate_1year,
                    'upper_1year': upper_1year,
                    'lower_1year': lower_1year,
                    'pct_change_1year': pct_change_1year,
                    'estimate_5year': estimate_5year,
                    'upper_5year': upper_5year,
                    'lower_5year': lower_5year,
                    'gross_profit_1year': gross_profit_1year, 
                    'pct_change_5year': pct_change_5year, 
                    'gross_profit_5year': gross_profit_5year}
    return zipcode_dict

In [18]:
def prophet_master(df, start, stop):
    """Puts together all the previous functions into one function that will return a list of dictionaries with the 
    results of the prophet predictions. Can control with start/stop."""
    preds = []
    for i in range(start, stop):
        zipcode = get_zipcode(df, i)
        forecast = Prophet_analysis(one_zipcode(df, i))
        combined = combine_dataframes(one_zipcode(df, i), forecast)
        preds.append(make_dict(combined, zipcode))
    return preds

In [23]:
#Suggestion: each person run approx. 4900 zipcodes:
#Brent: 0-4900
#Karen: 4901-9801
#Wyatt: 9802-14723

#Assign to new variable, then pickle the variable with this code:

In [25]:
# with open('prophet_lists_yourname', 'wb') as p:
#     pickle.dump(variable, p)

In [30]:
wyatt = prophet_master(df, 9802, 14723)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

...

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [32]:
with open('prophet_lists_wyatt', 'wb') as p:
    pickle.dump(wyatt, p)